In [ ]:
import numpy as np
import copy
import pickle

from enum import Enum
import sys

BOARD_ROW  = 3
BOARD_COL  = 3
WIN_LENGTH = 3

# Learning iterations
MAX_ITE = 0
Vinit = 4.0

MARKERNUMX =  1
MARKERNUMO = -1

class GAMESTATUS(Enum):
    X_WINS  =  1
    O_WINS  = -1
    DRAW    =  2
    ONGOING =  0


In [ ]:
# Utilities

# Find the player with the right to move based on the sum of the table, 1 is 'x', 0 is 'o'
def currentPlayer(state):
    return 'x' if (sum(state) == 0) else 'o' # player x or player o

# Find the player's number representing the mark, 1 for player 1 and -1 for player 0
def markerNumOnBoard(player):
    return MARKERNUMX if player == 'x' else MARKERNUMO

# Check if there is a streak of W values of xs or os in 1D arr
def checkConsecutive(arr, W, player):
    # Should replace o first because it is -1 so -111 will be wrongly considered a win for x
    strarr = "".join(map(str, arr)).replace(str(MARKERNUMO), 'o').replace(str(MARKERNUMX), 'x')
    if player*W in strarr:
        return True
    return False

# Check if the board is in terminal condition
def terminalCheck(state):

    board = np.array(state).reshape((BOARD_ROW, BOARD_COL))
    
    # Check the row and columns
    for idx in range(BOARD_ROW):
        if checkConsecutive(board[idx, :], WIN_LENGTH, 'x'):
            return GAMESTATUS.X_WINS
        if checkConsecutive(board[idx, :], WIN_LENGTH, 'o'):
            return GAMESTATUS.O_WINS
    for idx in range(BOARD_COL):
        if checkConsecutive(board[:, idx], WIN_LENGTH, 'x'):
            return GAMESTATUS.X_WINS
        if checkConsecutive(board[:, idx], WIN_LENGTH, 'o'):
            return GAMESTATUS.O_WINS

    # Check all diagonals (↘ and ↙) of length ≥ W
    for d in range(-BOARD_ROW, BOARD_COL):  # Diagonal offsets
        if checkConsecutive(np.diagonal(board, offset=d), WIN_LENGTH, 'x'):
            return GAMESTATUS.X_WINS
        if checkConsecutive(np.diagonal(board, offset=d), WIN_LENGTH, 'o'):
            return GAMESTATUS.O_WINS
        
    # Check all diagonals (↘ and ↙) of length ≥ W
    for d in range(-BOARD_ROW, BOARD_COL):  # Diagonal offsets
        if checkConsecutive(np.diagonal(np.fliplr(board), offset=d), WIN_LENGTH, 'x'):
            return GAMESTATUS.X_WINS
        if checkConsecutive(np.diagonal(np.fliplr(board), offset=d), WIN_LENGTH, 'o'):
            return GAMESTATUS.O_WINS

    # Check if the board is full (draw)
    if not np.any(board == 0):
        return GAMESTATUS.DRAW

    return GAMESTATUS.ONGOING

# Find the set of actions that can be taken from current state
def findActionSet(state):
    return [cidx for cidx, cellvalue in enumerate(state) if cellvalue == 0]

# Apply the action on the current state and get the new state
def applyAction(state, action):
    next_state = copy.copy(state)
    next_state[action] = markerNumOnBoard(currentPlayer(state))
    assert(next_state != state)
    return next_state

# Determine the reward from the state
def getReward(state, player):
    
    done = terminalCheck(state)
    
    if done == GAMESTATUS.ONGOING or done == GAMESTATUS.DRAW:
        return 0
    
    if (done == GAMESTATUS.X_WINS and player == 'x')\
    or (done == GAMESTATUS.O_WINS and player == 'o'):
        return 1
    
    if (done == GAMESTATUS.X_WINS and player == 'o')\
    or (done == GAMESTATUS.O_WINS and player == 'x'):
        return -1
    
    assert(False)    # Just make sure that we do not forget any logic here.
    return 0

# Find the set of states s' that the board will transition to after the agent takes an action
def possibleNextStateIdx(states, state, action):

    # Create a list of possible states that may occur after 'state'
    nextPossibleStateIdx = []

    # Apply the action and determine the possible next state
    next_state = applyAction(state, action)

    # Check the terminality of the 'next_state'
    done = terminalCheck(next_state)

    # If 'next_state' is terminal, add the 'next_state' to the list and return
    if done != GAMESTATUS.ONGOING:
        nextPossibleStateIdx.append(states.index(next_state))
        return nextPossibleStateIdx

    # If the state is not terminal, find the 'legal_opponent_actions' from the 'next state',
    # apply each action in 'legal_opponent_actions' to 'next_state' and add 'next_next_state' to the list
    if done == GAMESTATUS.ONGOING:
        legal_opponent_actions = findActionSet(next_state)
        for action in legal_opponent_actions:
            next_next_state = applyAction(next_state, action)
            nextPossibleStateIdx.append(states.index(next_next_state))

    # Return the next possible state
    return nextPossibleStateIdx

In [ ]:
# Create the value tables for player x and player o

import copy
import pickle

# Initialize the memories
state = [0 for idx in range(BOARD_ROW*BOARD_COL)]
Vx = Vinit
Vo = Vinit

memory = [[state], [Vx], [Vo]]

def TabulateStates(state):
    # Find all the possible actions from the current states
    actionSet = findActionSet(state)
    # Apply the action, then look further into the futher action
    for action in actionSet:
        new_state = applyAction(state, action)
        if new_state not in memory[0]:
            done = terminalCheck(new_state)
            # If the state is not terminal, go further till hitting a terminal state
            if done == GAMESTATUS.ONGOING:
                Vx, Vo = Vinit, Vinit
                TabulateStates(new_state)
            # If it's a win, reward 1 to winner and -1 to loser
            elif done == GAMESTATUS.X_WINS:
                Vx, Vo =  1, -1
            elif done == GAMESTATUS.O_WINS:
                Vx, Vo = -1,  1
            # If it's a draw, reward 0 to both
            elif done == GAMESTATUS.DRAW:
                Vx, Vo =  0,  0

            # code = "".join(map(str, new_state[:]))
            # code = code.replace('-1', 'B')
            # code = code.replace('1', 'A')
            # print("adding", code, done)

            memory[0].append(new_state)
            memory[1].append(Vx)
            memory[2].append(Vo)

TabulateStates(state)

print('States successfully tabulated with initial values!')
print('Size of memory: ', len(memory[0]))


In [ ]:
# Value iteration for best strategy against a random player


# with open('state_value_init.pkl', 'rb') as file:
#     memory = pickle.load(file)

states = memory[0]
Vx = memory[1]
Vo = memory[2]

gamma = 0.9

# Iterate for 5 times
for k in range(MAX_ITE):
    
    # Vx_k = Vx
    # Vo_k = Vo

    # Rate of change during each iteration
    dVmax = -np.inf

    # Iterate over the states
    for state in states:

        # Check the terminal category
        done = terminalCheck(state)

        # If the state is a terminal state, no need to update, otherwise perform value iteration
        if done != GAMESTATUS.ONGOING:
            continue
        
        # Index of the state under evaluation in the table
        sidx = states.index(state)

        # Determine the player's turn
        player = currentPlayer(state)

        # Find the legal actions at this state
        legal_actions = findActionSet(state)
        
        # Create a buffer for state-action value
        Qsa_buf = []

        # Find the state-action value for each action
        for action in legal_actions:

            # Identify the next outcomes
            nsidx = possibleNextStateIdx(states, state, action)

            # Probability of opponent making a counter move
            p = 1 / len(nsidx)
            Qsa = 0

            # Calculate the state-action value as an expectation
            for idx in nsidx:
                next_state = states[idx]
                value = Vx[idx] if player == 'x' else Vo[idx]
                # print(state, legal_actions, action, next_state)
                reward = getReward(next_state, player)
                Qsa += p * (reward + gamma * value)

            # Store that state-action value
            Qsa_buf.append(round(Qsa, 3))

        # Find the highest state-action value
        maxQsa = max(Qsa_buf)

        if player == 'x':
            dVmax = max(dVmax, abs(Vx[sidx] - maxQsa))
            Vx[sidx] = maxQsa
        else:
            dVmax = max(dVmax, abs(Vo[sidx] - maxQsa))
            Vo[sidx] = maxQsa

    print(f'Iteration {k}, change: {dVmax:.2f}')

state_action = [states, Vx, Vo]
with open('state_value_init.pkl', 'wb') as file:
    pickle.dump(state_action, file)

print('Successfully iterated over the states until convergence!')
print('Enjoy it!')


In [ ]:
#######################################################################
#
# Tic-Tac-Toe interactive environment
#
#######################################################################

class Game():

    def __init__(self, debug=False):
        self.done = GAMESTATUS.ONGOING                   # 0 for not done, 1 for X wins, -1 for O wins, -2 for illegal move, 2 for draw
        self.debug = debug

    def _draw(self):
        board_matrix = np.array(self.board).reshape((BOARD_ROW, BOARD_COL))
        # p1: x  p2: o
        for i in range(0, BOARD_ROW):
            print('----' * BOARD_COL + '-')
            out = '|'
            for j in range(0, BOARD_COL):
                if board_matrix[i, j] == 1:
                    token = '\033[32m X \033[0m'
                if board_matrix[i, j] == -1:
                    token = '\033[31m O \033[0m'
                if board_matrix[i, j] == 0:
                    token = f'{i*BOARD_ROW + j:2d} '
                out += token + '| '
            print(out)
        print('----' * BOARD_COL + '-')

    def _computeDone(self):
        self.done = terminalCheck(self.board)


    def _computeReward(self):
      return 0

    def step(self, action, player_who_acts):
        
        assert(currentPlayer(self.board) == player_who_acts)
        
        # Check for the illegal move
        if self.board[action] != 0:
            self.done = -2
        else:
            self.board[action] = markerNumOnBoard(player_who_acts)
            self._computeDone()

        self._computeReward()

        if self.debug:
            self._draw()

        return self.board, 0, self.done

    def reset(self):
        self.board = [0 for idx in range(BOARD_ROW*BOARD_COL)]
        if self.debug:
            self._draw()
        return self.board


In [ ]:
#######################################################################
#
# A simple human-AI game interface (needs to be improved :))
#
#######################################################################

# import tictactoe
import random
import pickle
# from utils import *
import time


print('\n\TicTacToe by Value Iteration!')
print('Good luck!')

def getPlayerChoice():
    human_player = None
    count = 0
    # Initialize the game
    while(human_player is None):
        choice = input(f'\nAttempt#{count}. X will go first. Please select O/X:\n\t O. Play as O. \n\t X. Play as X.\n')
        if choice.lower() == 'o':
            human_player = 'o'
        elif choice.lower() == 'x':
            human_player = 'x'
        else:
            print("Wrong choice, chose 'O/o' or 'X/x'.")
        count += 1
        if count > 5:
            break
    return human_player

def getHumanMove(state):
    
    legal_actions = findActionSet(state)
    
    count = 0
    print(f"Try {count}. Legal actions for human: {legal_actions}")
    while True:
        human_move = None
        try:
            human_move = int(input('Please make a move: '))
        except ValueError:
            human_move = None

        if human_move not in legal_actions:
            print(f"Attempt#{count}. Illegal move. Please chose in {legal_actions}")
        else:
            print(f"Human chooses action {human_move}")
            return human_move
        count += 1
        if count > 5:
            print("Exiting")
            sys.exit()  # Shutdown the kernel

    return legal_actions[0]

def policy(state, ai_player):
    legal_actions = findActionSet(state)
    Qsa_buf = []
    for action in legal_actions:
        nsidx = possibleNextStateIdx(states, state, action)
        p = 1 / len(nsidx)
        Qsa = 0
        for sidx in nsidx:
            ai_statevalue = Vo[sidx] if ai_player == 'o' else Vx[sidx]
            reward = getReward(states[sidx], ai_player)
            Qsa += p * (reward + gamma * ai_statevalue)
        Qsa_buf.append(round(Qsa, 3))
    action = legal_actions[Qsa_buf.index(max(Qsa_buf))]
    return action, legal_actions, Qsa_buf
    

# Create the environment
env = Game(debug=True)

# Export the memories as list
# with open(f'state_value_{BOARD_ROW}x{BOARD_COL}.pkl', 'rb') as file:
#     state_action = pickle.load(file)

states = state_action[0]
Vx = state_action[1]
Vo = state_action[2]
score_table = [0, 0]    # human, AI

while True:

    # Prompt human's preferred symbol
    human_player = getPlayerChoice()
    ai_player = 'x' if human_player == 'o' else 'o'

    init = True
    done = GAMESTATUS.ONGOING
    state = env.reset()

    # Start playing
    while done == GAMESTATUS.ONGOING:
        
        # If human goes first
        if (init and human_player == 'x'):
            init = False
            human_move = getHumanMove(state)
            state, reward, done = env.step(human_move, player_who_acts = human_player)
            continue
        elif init:  # If AI goes first
            init = False
            action, legal_actions, Qsa_buf = policy(state, ai_player)
            print(f'AI chooses action {action} ...')
            state, reward, done = env.step(action, player_who_acts = ai_player)
            human_move = getHumanMove(state)
            state, reward, done = env.step(human_move, player_who_acts = human_player)

        print('VI enacting policy...')
        action, legal_actions, Qsa_buf = policy(state, ai_player)
        print('\tlegal acttion', legal_actions)
        print('\taction values', Qsa_buf)
        print('AI choses action:', action)
        
        # Apply the action
        state, reward, done = env.step(action, player_who_acts = ai_player)
        
        # Human moves
        if done == GAMESTATUS.ONGOING:
            human_move = getHumanMove(state)
            state, reward, done = env.step(human_move, player_who_acts = human_player)

    victor = None
    if done == GAMESTATUS.X_WINS:
        victor = 'x'
    elif done == GAMESTATUS.O_WINS:
        victor = 'o'

    if victor == human_player:
        score_table[0] += 1
    elif done != GAMESTATUS.DRAW:
        score_table[1] += 1

    print("\n\nGame Over!")
    print('Human Score: ', score_table[0])
    print('AI Score: ' , score_table[1], '\n\n')
